1.**导入数据**

In [24]:
import torch
from torchtext import data
from torchtext.legacy.data import Field,LabelField,BucketIterator
seed=1111
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

TEXT = Field(tokenize='spacy',tokenizer_language='en_core_web_sm')
LABEL = LabelField(dtype=torch.float)


In [25]:
from torchtext.legacy import datasets
train_data,test_data = datasets.IMDB.splits(TEXT,LABEL)

In [26]:
 #查看数据集
vars(train_data.examples[0])

{'label': 'pos',
 'text': ['Amazing',
  'documentary',
  '.',
  'Saw',
  'it',
  'on',
  'original',
  'airdate',
  'and',
  'on',
  'DVD',
  'a',
  'few',
  'times',
  'in',
  'the',
  'last',
  'few',
  'years',
  '.',
  'I',
  'was',
  'shocked',
  'that',
  'it',
  'was',
  "n't",
  'even',
  'nominated',
  'for',
  'a',
  'Best',
  'Documentary',
  'Oscar',
  'for',
  '2002',
  ',',
  'the',
  'year',
  'it',
  'was',
  'released',
  '.',
  'No',
  'other',
  'documentary',
  'even',
  'comes',
  'close.<br',
  '/><br',
  '/>It',
  'was',
  'on',
  'TV',
  'recently',
  'for',
  'the',
  '5th',
  'anniversary',
  ',',
  'but',
  'I',
  'missed',
  'the',
  'added',
  '"',
  'where',
  'are',
  'they',
  'now',
  '"',
  'segment',
  'at',
  'the',
  'end',
  ',',
  'except',
  'I',
  'did',
  'catch',
  'that',
  'tony',
  'now',
  'works',
  'for',
  'the',
  'hazmat',
  'unit.<br',
  '/><br',
  "/>I've",
  'seen',
  'criticism',
  'on',
  'documentary',
  'film',
  '-',
  'making

2.**切分数据集**

In [27]:
import random
#切分数据集 默认0.7
train_data, valid_data = train_data.split(random_state=random.seed(seed))
print(f'训练集数量: {len(train_data)}')
print(f'验证集数量: {len(valid_data)}')
print(f'测试集合数量: {len(test_data)}')

训练集数量: 17500
验证集数量: 7500
测试集合数量: 25000


3.**训练集构造词汇表**

In [28]:
#使用预训练glove模型，更新之后作为词向量
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data) 

In [29]:
print(f"词汇表中词数: {len(TEXT.vocab)}")
print(f"label数: {len(LABEL.vocab)}")

词汇表中词数: 25002
label数: 2


4.**创建迭代器**

In [30]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data),batch_size=BATCH_SIZE,device=device)

In [31]:
print(next(iter(train_iterator)).label.shape)
print(next(iter(train_iterator)).text.shape)  

torch.Size([64])
torch.Size([986, 64])


In [32]:
batch = next(iter(train_iterator))
print(batch.text.shape) 

torch.Size([1153, 64])


**5.构造Word Averaging模型**

In [33]:
import torch.nn as nn
import torch.nn.functional as F

class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx) 
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text): 
        embedded = self.embedding(text)  
        embedded = embedded.permute(1, 0, 2)
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)   

        return self.fc(pooled)
VOCAB_SZIE = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1 
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] 

model = WordAVGModel(VOCAB_SZIE, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

6.**初始化参数**

In [34]:
#用glove.6B.100d的参数作为初始化参数
pretrained_embeddings = TEXT.vocab.vectors 
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.1065,  0.1614, -0.6850,  ..., -0.9348, -0.4778,  1.1286],
        [ 0.2091,  0.2932,  0.3151,  ...,  1.1860, -1.5726,  1.1354],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2572,  0.4990,  0.7320,  ..., -0.6175, -0.5058,  0.4684],
        [ 0.0701,  0.4941,  0.1676,  ...,  0.1009, -0.4823,  0.4224],
        [-0.2552, -0.7599, -1.0630,  ...,  0.1064, -0.8649, -1.0150]])

In [35]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) 
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

7.**模型训练**

In [36]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters()) 
loss_fn = nn.BCEWithLogitsLoss()
model = model.to(device) 
loss_fn = loss_fn.to(device) 

In [37]:
def binary_accuracy(preds, y): 
    '''计算准确度，即预测和实际标签的相匹配的个数'''
    rounded_preds = torch.round(torch.sigmoid(preds)) 
    correct = (rounded_preds == y).float()
    acc = correct.sum()/len(correct)
    return acc


def train(model, iterator, optimizer, loss_fn):
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.train() 
    
    for batch in iterator: 
        optimizer.zero_grad() 
        preds = model(batch.text).squeeze(1)
        loss = loss_fn(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
             
        loss.backward() 
        optimizer.step() 
        
        epoch_loss += loss.item() * len(batch.label)

        
        epoch_acc += acc.item() * len(batch.label)

        
        total_len += len(batch.label)

        
    return epoch_loss / total_len, epoch_acc / total_len

In [38]:
def evaluate(model, iterator, loss_fn):
    epoch_loss = 0
    epoch_acc = 0
    total_len = 0
    model.eval()

    with torch.no_grad():  
        for batch in iterator: 
            
            preds = model(batch.text).squeeze(1)
            loss = loss_fn(preds, batch.label)
            acc = binary_accuracy(preds, batch.label)
            
            epoch_loss += loss.item() * len(batch.label)
            epoch_acc += acc.item() * len(batch.label)
            total_len += len(batch.label)
    model.train()
    return epoch_loss / total_len, epoch_acc / total_len
import time 

def epoch_time(start_time, end_time): 
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
N_EPOCHS = 20
best_valid_loss = float('inf') 

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn) 
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss: 
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'wordavg-model.pth')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.685 | Train Acc: 58.59%
	 Val. Loss: 0.627 |  Val. Acc: 67.49%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.644 | Train Acc: 75.45%
	 Val. Loss: 0.497 |  Val. Acc: 77.37%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.570 | Train Acc: 80.12%
	 Val. Loss: 0.430 |  Val. Acc: 80.77%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.497 | Train Acc: 83.47%
	 Val. Loss: 0.402 |  Val. Acc: 83.67%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.436 | Train Acc: 86.12%
	 Val. Loss: 0.403 |  Val. Acc: 84.87%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.385 | Train Acc: 88.06%
	 Val. Loss: 0.411 |  Val. Acc: 85.99%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 0.345 | Train Acc: 89.19%
	 Val. Loss: 0.419 |  Val. Acc: 86.65%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 0.317 | Train Acc: 90.29%
	 Val. Loss: 0.438 |  Val. Acc: 87.12%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.290 | Train Acc: 91.02%
	 Val. Loss: 0.454 |  Val. Acc: 87.55%
Epoch: 10 | Epoch Time: 0m 7

8.**预测**

In [19]:
import spacy  
nlp = spacy.load('en_core_web_sm') 

def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    print(tokenized)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized] 
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) 
    pred = torch.sigmoid(model(tensor))
    return pred.item()


In [21]:
predict_sentiment("I love This film")

['I', 'love', 'This', 'film', 'bad']


0.9999997615814209